In [1]:
import numpy as np 
from scipy.stats import multivariate_normal
import Estimateurs

On commence par tirer le $\theta$* ($\in \mathbb{R}$) qui sera le paramètre que l'on cherchera à estimer par la suite. 

A la manière de la génération des données dans l'article Rainforth et al. (2018) : $\theta^* \sim \mathcal{N}(0, 1)$

In [2]:
#theta_true = np.random.multivariate_normal(np.zeros(20), np.identity(20))
theta_true = np.random.normal(0, 1)
print(f"La valeur de theta à estimer est {int(theta_true*100)/100}")

La valeur de theta à estimer est 0.58


Désormais, on va tirer un échantillon $((Z_i, X_i))_i \stackrel{iid}{\sim} \mathcal{N}(\boldsymbol{z}|\theta, \boldsymbol{I}) * \mathcal{N}(\boldsymbol{x}|\boldsymbol{z}, \boldsymbol{I})$. 


In [6]:
## On se donne notre échantillon x

x, _ = Estimateurs.joint_probability(theta_true)

print(f"L'échantillon x observé est :\n \n x = {x}")
print(x.shape)

L'échantillon x observé est :
 
 x = [ 2.20422981  2.26469783  0.85109594 -0.72115318  0.74210777  2.58821114
 -0.04561904 -0.65143135 -1.66634061 -0.92748446 -1.14232242  0.46471566
 -0.61563554  0.08950375  2.2530738   0.53737847  2.12590487  3.57317853
  1.56709784 -1.29055781]
(20,)


Ceci nous suffit pour tester notre classe Estimateurs dont le code se trouve dans le fichier Estimateurs.py

In [4]:
estimateur = Estimateurs.Estimateurs(x, theta_true, 0.6)

In [5]:
estimateur.grad_ML_RR(theta_star=theta_true, n_simulations=100)

ValueError: cannot reshape array of size 400 into shape (1,1)